In [9]:
import numpy as np
import re
%pip install scikeras
from scikeras.wrappers import KerasClassifier
from typing import Tuple, List, Union, Optional
from networkx import scale_free_graph
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from transformers import BertTokenizer
import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
import datetime
from scipy.spatial.distance import cosine
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
import ast


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import re
%pip install scikeras
from scikeras.wrappers import KerasClassifier
from typing import Tuple, List, Union, Optional
from networkx import scale_free_graph
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier

import ast
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from transformers import BertTokenizer
import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
import datetime
from scipy.spatial.distance import cosine
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer
import json
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, KFold

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
from sklearn.decomposition import PCA

from sklearn.metrics.pairwise import cosine_similarity
import ast
import datetime
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
class ClassicalModels:
    
    def __init__(self, data: pd.DataFrame, target: str, max_length: int = 500):
        self.data = data
        self.target = target
        self.max_length = max_length
        self.model: Optional[model] = None
        self.batch_size = 8
        self.epochs=50
        
        self.X_cosine=None
        self.y=None
        
        
        # Configure logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    @staticmethod
    def remove_java_string(text: str) -> str:
        return text.replace("```java", "").replace("```", "")

    def pad_sequence(self, tokens: List[int], max_length: Optional[int] = None) -> np.ndarray:
        max_length = max_length or self.max_length
        tokens = np.array(tokens)
        
        if len(tokens) < max_length:
            return np.pad(tokens, (0, max_length - len(tokens)), mode='constant')
        return tokens[:max_length]

    def preprocess_data(self, df: pd.DataFrame) :

        # Clean data
        if 'Unnamed: 0' in df.columns:
            df.drop(['Unnamed: 0'], axis=1, inplace=True)
            
        print(f"Processing DataFrame with shape: {df.shape}")
            
        # Clean Java strings
        df['java_translation'] = df['java_translation'].apply(
                lambda x: self.remove_java_string(str(x)) if isinstance(x, list) else self.remove_java_string(x)
            )
            
        # Process token sequences
        df['java_features'] = df['java_tokens'].apply(
                lambda x: self.pad_sequence(eval(x) if isinstance(x, str) else x)
            )
        df['python_features'] = df['python_tokens'].apply(
                lambda x: self.pad_sequence(eval(x) if isinstance(x, str) else x)
            )
        # Prepare features and target
        features = np.array(df['java_features'].tolist() + df['python_features'].tolist())
        target = np.array(df['Is_Equal'].tolist() * 2)
            
        #Scale Features
        scaler=StandardScaler()
        features=scaler.fit_transform(features)
            

        return features, target, df
            



    def tokenize_data(self, df: pd.DataFrame):

        # Tokenize Java code
        tokenizer= BertTokenizer.from_pretrained('bert-base-cased')
        df['java_tokens'] = df['java_translation'].apply(
                lambda x: tokenizer.encode(x, add_special_tokens=True)
            )
            
            
        df['python_tokens'] = df['original_code'].apply(
                lambda x: tokenizer.encode(x, add_special_tokens=True)
            )
            
        # Convert target to int
        df['Is_Equal'] = df['Is_Equal'].astype(int)
        return df


    def train_random_forest(self,features,target):

        X_train, X_test, y_train, y_test = train_test_split(
                features, target, test_size=0.2, random_state=42
            )
        model=RandomForestClassifier(
                    criterion='entropy',
                    max_depth=8,
                    max_features='sqrt',
                    min_samples_leaf=8,
                    min_samples_split=10
                )
            
        model.fit(X_train, y_train)
        self.model = model
        y_pred = model.predict(X_test)
            

        print(f"RF  Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"RF Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
        print(f"Rf Classification Report:\n{classification_report(y_test, y_pred)}")
        print(f"RF Train Score: {model.score(X_train, y_train):.4f}")
        print(f"RF Test Score: {model.score(X_test, y_test):.4f}")
            
        return model

    def neural_network(self,features,target) :
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
        model = Sequential([
            Dense(32, input_shape=(self.max_length,), activation='relu'),
            BatchNormalization(),
            Dropout(0.1),
            Dense(32, activation='relu'),
            BatchNormalization(),
            Dropout(0.1),
            Dense(32, activation='relu'),
            
            Dense(1, activation='sigmoid')
        ])

        model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )


        lr_scheduler = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=0.00001
            )

        print("Starting neural network training...")
        history = model.fit(
                X_train,
                y_train,
                epochs=self.epochs,
                batch_size=self.batch_size,
                validation_data=(X_test, y_test),
                callbacks=[lr_scheduler]
            )


        train_loss, train_accuracy = model.evaluate(X_train, y_train)
        test_loss, test_accuracy = model.evaluate(X_test, y_test)

        print(f"NN Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"NN Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


        y_pred = model.predict(X_test) > 0.5
        print(f"NN Classification Report:\n{classification_report(y_test, y_pred)}")
        print(f"NN Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")

        return model    
    
    def svm_preprocessing(self,data):
        max_length = max(
            max(len(x) for x in data['python_tokens']),
            max(len(x) for x in data['java_tokens'])
        )

        # Pad tokens
        data['python_tokens'] = data['python_tokens'].apply(
            lambda x: np.pad(x, (0, max_length - len(x)), mode='constant')[:max_length]
        )
        data['java_tokens'] = data['java_tokens'].apply(
            lambda x: np.pad(x, (0, max_length - len(x)), mode='constant')[:max_length]
        )

        # Create feature matrix
        X = np.array([np.abs(np.array(python) - np.array(java)) for python, java in zip(data['python_tokens'], data['java_tokens'])])
        y = data['Is_Equal']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        return X_train,X_test,y_train,y_test
    
    def svm_model(self,X_train, X_test, y_train, y_test):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        smote = SMOTE(random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        model = SVC(C=1, gamma='auto', kernel='rbf')
        model.fit(X_train, y_train)

        train_accuracy = model.score(X_train, y_train)
        test_accuracy = model.score(X_test, y_test)

        print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
        print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
        return model
        
    def knn_preprocessing(sekf,data):

        fixed_length = 512
        java_tokens_padded = np.array([np.pad(seq, (0, max(0, fixed_length - len(seq))), mode='constant')[:fixed_length] for seq in data['java_tokens']])
        python_tokens_padded = np.array([np.pad(seq, (0, max(0, fixed_length - len(seq))), mode='constant')[:fixed_length] for seq in data['python_tokens']])
        X = np.linalg.norm(java_tokens_padded - python_tokens_padded, axis=1).reshape(-1, 1)
        y = data['Is_Equal'].values 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        return  X_train, X_test, y_train, y_test
    
    def knn_model(self,X_train, X_test, y_train, y_test):
        k = 3
        knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean', weights='uniform')
        knn.fit(X_train, y_train)
        y_train_pred = knn.predict(X_train)
        y_test_pred = knn.predict(X_test)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
        print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")
        return knn


\

    

data=pd.read_csv('dataa.csv')
models=ClassicalModels(data, 'Is_Equal')
models.tokenize_data(data)
features, target,df = models.preprocess_data(data)




[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Token indices sequence length is longer than the specified maximum sequence length for this model (580 > 512). Running this sequence through the model will result in indexing errors
Processing DataFrame with shape: (477, 5)


In [15]:
# # # Train other models
models.train_random_forest(features, target)

RF  Accuracy: 0.6021
RF Confusion Matrix:
[[41 53]
 [23 74]]
Rf Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.44      0.52        94
           1       0.58      0.76      0.66        97

    accuracy                           0.60       191
   macro avg       0.61      0.60      0.59       191
weighted avg       0.61      0.60      0.59       191

RF Train Score: 0.9279
RF Test Score: 0.6021


RandomForestClassifier(criterion='entropy', max_depth=8, min_samples_leaf=8,
                       min_samples_split=10)

In [17]:
models.neural_network(features, target)

/root/venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Starting neural network training...
Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4884 - loss: 0.7239 - val_accuracy: 0.5445 - val_loss: 0.7033 - learning_rate: 0.0010
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6364 - loss: 0.6291 - val_accuracy: 0.5340 - val_loss: 0.7220 - learning_rate: 0.0010
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7150 - loss: 0.5756 - val_accuracy: 0.5445 - val_loss: 0.7325 - learning_rate: 0.0010
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7899 - loss: 0.5034 - val_accuracy: 0.5497 - val_loss: 0.7424 - learning_rate: 0.0010
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━

<Sequential name=sequential, built=True>

In [24]:
X_train, X_test, y_train, y_test = models.svm_preprocessing(df)
models.svm_model(X_train, X_test, y_train, y_test)

/root/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
Train Accuracy: 79.00%
Test Accuracy: 52.08%


SVC(C=1, gamma='auto')

In [38]:

X_train, X_test, y_train, y_test =models.knn_preprocessing(df)
models.knn_model(X_train, X_test, y_train, y_test)



Training Accuracy: 74.28%
Testing Accuracy: 64.58%


In [ ]:
import pickle

pickle.dump(ensemblemodel, open('ensemblemodel.pkl', 'wb'))



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2fe094d0-9fb3-4318-be9c-48cc4ca88a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>